In [8]:
import json
import os
from sklearn.ensemble import BaggingRegressor
from sklearn.neural_network import MLPRegressor
from multi_basic_estimator_regressor import MultiBasicEstimatorRegressor, mber_save, mber_load
from rppg_dataset import RPPGDataset
from torch.utils.data import DataLoader
import torch
import sklearn.metrics
import numpy as np
from multi_basic_estimator import MultiBasicEstimator
from basic_estimator import rppg_heart_rate

In [9]:
# Load project information.
project_root = '../..'
project_file_name = 'project_info.json'
if not os.path.exists(project_file_name):
    os.chdir(project_root)
with open(project_file_name, 'r') as f:
    info = json.load(f)

In [10]:
model_output_path = info['mber_path']
sampling_rate = info['sampling_rate']
data_path = info['data_path']

print(model_output_path)

resources/models/mber.pickle


In [11]:
data_root = os.path.join(data_path, 'generation1')
csv_path = os.path.join(data_path, 'generation1', 'data.csv')

dataset = RPPGDataset(csv_path, data_root)

c = 5100
data_loader = DataLoader(dataset, batch_size=c)
data_loader_iter = iter(data_loader)
X, y = next(data_loader_iter)
y = np.array([rppg_heart_rate(s, sampling_rate) for s in y.numpy()])
X_train, y_train = X[:1000], y[:1000]
X_test, y_test = X[c-30:], y[c-30:]

#X_train = torch.flatten(X_train, start_dim=1)

n_regions = 4 # Data determines

base_regressor = MLPRegressor(random_state=1, hidden_layer_sizes = 5, max_iter=500)
ensemble_regressor = BaggingRegressor(base_regressor, 4)
multi_basic_estimator_regressor = MultiBasicEstimatorRegressor(ensemble_regressor, n_regions, sampling_rate)

In [12]:
multi_basic_estimator_regressor.fit(X_train, y_train)

/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ignoring n_components with whiten=False.")
/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ignoring n_components with whiten=False.")
/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ignoring n_components with whiten=False.")
/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ignoring n_components with whiten=False.")
/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ign

torch.Size([1000, 4])
(1000,)


In [13]:
mber_save(multi_basic_estimator_regressor, model_output_path)

In [14]:
mbe = MultiBasicEstimator(4, sampling_rate)
model1 = lambda A : np.median(mbe(A), axis=1)
model2 = mber_load(model_output_path)

mses = []
res = []
for model in [model1, model2]:
#X_test, y_test = next(data_loader_iter)
    y_trues = y_test
    y_preds = model(X_test)
    mse = sklearn.metrics.mean_squared_error(y_trues, y_preds)
    mses.append(mse)
    res.append(y_preds.reshape(30))
res.append(y_trues)
for mse in mses:
    print(mse)
print(np.array(res))

/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ignoring n_components with whiten=False.")
/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ignoring n_components with whiten=False.")
/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ignoring n_components with whiten=False.")
/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ignoring n_components with whiten=False.")
/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ign

0.028531311880886104
0.006544136764506435
[[0.51085567 0.51085567 0.51085567 0.38314176 0.12771392 0.51085567
  0.12771392 0.51085567 0.51085567 0.38314176 0.12771392 0.51085567
  0.51085567 0.51085567 0.38314176 0.12771392 0.51085567 0.51085567
  0.51085567 0.38314176 0.51085567 0.12771392 0.38314176 0.51085567
  0.12771392 0.51085567 0.38314176 0.51085567 0.38314176 0.38314176]
 [0.50031114 0.50031114 0.50031114 0.46834928 0.33674034 0.50031114
  0.33674034 0.50031114 0.50031114 0.46834928 0.33674034 0.50031114
  0.50031114 0.50031114 0.46834928 0.33674034 0.50031114 0.50031114
  0.50031114 0.46834928 0.50031114 0.33674034 0.46834928 0.50031114
  0.33674034 0.50031114 0.46834928 0.50031114 0.46834928 0.46834928]
 [0.44444444 0.44444444 0.44444444 0.44444444 0.44444444 0.55555556
  0.44444444 0.55555556 0.44444444 0.44444444 0.55555556 0.44444444
  0.44444444 0.55555556 0.44444444 0.44444444 0.44444444 0.44444444
  0.55555556 0.44444444 0.44444444 0.44444444 0.44444444 0.55555556
  0.

/home/frederik-v-b/.local/lib/python3.10/site-packages/sklearn/decomposition/_fastica.py:583: UserWarning: Ignoring n_components with whiten=False.
  warnings.warn("Ignoring n_components with whiten=False.")
